# 모델 선택 및 학습
## 모델 선택

In [ ]:
import os
import tensorflow as tf
# GPU 메모리 사용 점진적 할당 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [ ]:
# 테스트 환경 속 설정 변수
epochs = 20
batch_size = 32
validation_split = 0.2

In [ ]:
import numpy as np
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# 파일 경로 설정
train_file_path = 'Data/processed_train2.csv'
test_file_path = 'Data/processed_test2.csv'
train_data = pd.read_csv(train_file_path)
test_data = pd.read_csv(test_file_path)


# 텍스트 정제 함수
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # 다중 공백 제거
    text = re.sub(r'[^\w\s]', '', text)  # 특수 문자 제거
    return text.strip()

train_data['cleaned_conversation'] = train_data['conversation'].apply(clean_text)
train_data['text_length'] = train_data['cleaned_conversation'].apply(len)

# 토크나이저 설정
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['cleaned_conversation'])

# 시퀀스 변환 및 패딩
sequences = tokenizer.texts_to_sequences(train_data['cleaned_conversation'])
max_length = 300
X_data = pad_sequences(sequences, maxlen=max_length, padding='post')

# 레이블 인코딩
label_encoder = LabelEncoder()
y_data = label_encoder.fit_transform(train_data['class'])

# 결과 확인
print('토크나이저 단어 개수:', len(tokenizer.word_index))
print('X_data 크기:', X_data.shape)
print('y_data 크기:', y_data.shape)

In [ ]:
padded_sequences = X_data
labels = y_data

## 1. 기본 LSTM 모델

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense

# 입력 정의
input_layer = Input(shape=(max_length,), name='input')

# 임베딩 층
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128)(input_layer)

# LSTM 층
lstm_layer = LSTM(64)(embedding_layer)

# 출력층 정의
num_classes = 4
output_layer = Dense(num_classes, activation='softmax', name='output')(lstm_layer)

# 모델 생성
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 요약
model.summary()

In [ ]:
# 모델 학습
history = model.fit(
    padded_sequences, labels,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=validation_split
)

In [ ]:
model_name = 'model_bi'
model.save(f'Model/{model_name}.h5')

In [ ]:
from transformers import AutoTokenizer

def create_submission_file(model_name, test_data, tokenizer, label_encoder, max_length):
    # 토크나이저 타입 확인 및 시퀀스 변환
    if isinstance(tokenizer, AutoTokenizer):
        # Transformers 토크나이저 사용
        test_texts = test_data['text'].tolist()
        X_test = tokenizer.batch_encode_plus(
            test_texts,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_tensors='tf'
        )['input_ids']
    elif isinstance(tokenizer, Tokenizer):
        # Keras Tokenizer 사용
        X_test = tokenizer.texts_to_sequences(test_data['text'].tolist())
        X_test = pad_sequences(X_test, maxlen=max_length, padding='post')
    else:
        raise ValueError("지원되지 않는 토크나이저 타입입니다.")
    
    # 모델 로드
    model = tf.keras.models.load_model(f'{model_name}.h5')
    
    # 예측 생성
    test_predictions = model.predict(X_test)
    test_predicted_labels = tf.argmax(test_predictions, axis=1).numpy()
    
    # 라벨 디코딩
    test_predicted_labels = label_encoder.inverse_transform(test_predicted_labels)
    
    # 서브미션 파일 생성
    submission = pd.DataFrame({'index': test_data.index, 'class': test_predicted_labels})
    submission_filename = f'{model_name}_submission.csv'
    submission.to_csv(submission_filename, index=False)
    print(f'Submission file created: {submission_filename}')

create_submission_file(model_name, test_data, tokenizer, label_encoder, max_length)